In [1]:
# rcnn_experiments.ipynb – Cell 1
import torch
import torchvision.transforms as T
from torch.utils.data import DataLoader
from torchvision.ops import box_iou

from rcnn_dataset import FruitDetectionDataset
from rcnn_model import get_faster_rcnn_model

import os
from PIL import Image, ImageDraw

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", DEVICE)

# CHANGE THIS if you have a different number of fruit classes
NUM_FRUIT_CLASSES = 10
NUM_CLASSES = NUM_FRUIT_CLASSES + 1   # +1 for background

# Fill in your actual class names in order
CLASS_NAMES = [
    "background",
    "apple",
    "avocado",
    "banana",
    "kiwi",
    "lemon",
    "orange",
    "pear",
    "pomegranate",
    "strawberry",
    "watermelon",
]

Using device: cpu


In [2]:
train_imgs   = "../dataset/split/images/train"
train_labels = "../dataset/split/labels/train"

val_imgs     = "../dataset/split/images/val"
val_labels   = "../dataset/split/labels/val"

print("Train images dir:", train_imgs)
print("Train labels dir:", train_labels)
print("Val images dir:", val_imgs)
print("Val labels dir:", val_labels)

import torchvision.transforms as T
from torch.utils.data import DataLoader

def get_transforms(train=True):
    transforms = [T.ToTensor()]
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

def collate_fn(batch):
    return tuple(zip(*batch))

# create datasets
train_dataset = FruitDetectionDataset(
    images_dir=train_imgs,
    labels_dir=train_labels,
    transforms=get_transforms(train=True),
)

val_dataset = FruitDetectionDataset(
    images_dir=val_imgs,
    labels_dir=val_labels,
    transforms=get_transforms(train=False),
)

print("Train samples:", len(train_dataset))
print("Val samples:", len(val_dataset))

# data loaders
train_loader = DataLoader(
    train_dataset,
    batch_size=2,
    shuffle=True,
    num_workers=0,
    collate_fn=collate_fn,
)

val_loader = DataLoader(
    val_dataset,
    batch_size=2,
    shuffle=False,
    num_workers=0,
    collate_fn=collate_fn,
)


Train images dir: ../dataset/split/images/train
Train labels dir: ../dataset/split/labels/train
Val images dir: ../dataset/split/images/val
Val labels dir: ../dataset/split/labels/val
Train samples: 790
Val samples: 205


In [3]:
model = get_faster_rcnn_model(num_classes=NUM_CLASSES)
model.to(DEVICE)

params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(
    params,
    lr=0.005,
    momentum=0.9,
    weight_decay=0.0005,
)

lr_scheduler = torch.optim.lr_scheduler.StepLR(
    optimizer,
    step_size=5,
    gamma=0.1,
)

print("Model ready.")

Model ready.


In [4]:
def train_one_epoch(model, optimizer, data_loader, epoch):
    model.train()
    total_loss = 0.0

    for images, targets in data_loader:
        images = [img.to(DEVICE) for img in images]
        targets = [{k: v.to(DEVICE) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        total_loss += losses.item()

    avg_loss = total_loss / len(data_loader)
    print(f"Epoch {epoch}: train loss = {avg_loss:.4f}")
    return avg_loss


@torch.no_grad()
def evaluate_simple(model, data_loader):
    """
    Very simple evaluation: computes mean IoU between
    predicted boxes and ground-truth boxes.
    """
    model.eval()
    total_iou = 0.0
    n_boxes = 0

    for images, targets in data_loader:
        images = [img.to(DEVICE) for img in images]
        outputs = model(images)

        for out, tgt in zip(outputs, targets):
            if len(out["boxes"]) == 0 or len(tgt["boxes"]) == 0:
                continue

            ious = box_iou(out["boxes"].cpu(), tgt["boxes"])
            max_iou, _ = ious.max(dim=0)   # best IoU per GT box
            total_iou += max_iou.sum().item()
            n_boxes += len(max_iou)

    if n_boxes > 0:
        mean_iou = total_iou / n_boxes
        print(f"Mean IoU over GT boxes: {mean_iou:.3f}")
        return mean_iou
    else:
        print("No boxes to evaluate (check labels).")
        return 0.0

In [5]:
num_epochs = 10
train_losses = []
val_ious = []

for epoch in range(1, num_epochs + 1):
    loss = train_one_epoch(model, optimizer, train_loader, epoch)
    train_losses.append(loss)

    lr_scheduler.step()

    iou = evaluate_simple(model, val_loader)
    val_ious.append(iou)

c:\Users\moozh\AppData\Local\Programs\Python\Python311\Lib\site-packages\PIL\Image.py:1039: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch 1: train loss = 0.7410
Mean IoU over GT boxes: 0.784
Epoch 2: train loss = 0.5989
Mean IoU over GT boxes: 0.788
Epoch 3: train loss = 0.5544
Mean IoU over GT boxes: 0.809
Epoch 4: train loss = 0.5079
Mean IoU over GT boxes: 0.818
Epoch 5: train loss = 0.4956
Mean IoU over GT boxes: 0.802
Epoch 6: train loss = 0.4354
Mean IoU over GT boxes: 0.830
Epoch 7: train loss = 0.4040
Mean IoU over GT boxes: 0.828
Epoch 8: train loss = 0.3919
Mean IoU over GT boxes: 0.829
Epoch 9: train loss = 0.3809
Mean IoU over GT boxes: 0.831
Epoch 10: train loss = 0.3835
Mean IoU over GT boxes: 0.829


In [6]:
save_path = "../models/faster_rcnn_fruits.pth"
os.makedirs(os.path.dirname(save_path), exist_ok=True)

torch.save(model.state_dict(), save_path)
print("Saved model to:", save_path)

Saved model to: ../models/faster_rcnn_fruits.pth


In [7]:
test_imgs_dir = "../dataset/test_images/test"
output_dir = "../test_results/faster_rcnn"
os.makedirs(output_dir, exist_ok=True)

score_thresh = 0.5  # confidence threshold
transform = T.ToTensor()

model.eval()

for fname in os.listdir(test_imgs_dir):
    if not fname.lower().endswith((".jpg", ".jpeg", ".png")):
        continue

    img_path = os.path.join(test_imgs_dir, fname)
    img = Image.open(img_path).convert("RGB")
    img_tensor = transform(img).to(DEVICE)

    with torch.no_grad():
        output = model([img_tensor])[0]

    boxes = output["boxes"].cpu()
    scores = output["scores"].cpu()
    labels = output["labels"].cpu()

    draw = ImageDraw.Draw(img)

    for box, score, label in zip(boxes, scores, labels):
        if score < score_thresh:
            continue

        x1, y1, x2, y2 = box.tolist()
        cls_name = CLASS_NAMES[int(label)]

        # Draw box
        draw.rectangle([x1, y1, x2, y2], outline="red", width=3)

        # Draw text (class + score)
        text = f"{cls_name} {score:.2f}"
        draw.text((x1 + 3, y1 + 3), text, fill="yellow")

    out_path = os.path.join(output_dir, fname)
    img.save(out_path)
    print("Saved:", out_path)

Saved: ../test_results/faster_rcnn\apple_77.jpg
Saved: ../test_results/faster_rcnn\apple_78.jpg
Saved: ../test_results/faster_rcnn\apple_79.jpg
Saved: ../test_results/faster_rcnn\apple_80.jpg
Saved: ../test_results/faster_rcnn\apple_81.jpg
Saved: ../test_results/faster_rcnn\apple_82.jpg
Saved: ../test_results/faster_rcnn\apple_83.jpg
Saved: ../test_results/faster_rcnn\apple_84.jpg
Saved: ../test_results/faster_rcnn\apple_85.jpg
Saved: ../test_results/faster_rcnn\apple_86.jpg
Saved: ../test_results/faster_rcnn\apple_87.jpg
Saved: ../test_results/faster_rcnn\apple_88.jpg
Saved: ../test_results/faster_rcnn\apple_89.jpg
Saved: ../test_results/faster_rcnn\apple_90.jpg
Saved: ../test_results/faster_rcnn\apple_91.jpg
Saved: ../test_results/faster_rcnn\apple_92.jpg
Saved: ../test_results/faster_rcnn\apple_93.jpg
Saved: ../test_results/faster_rcnn\apple_94.jpg
Saved: ../test_results/faster_rcnn\apple_95.jpg
Saved: ../test_results/faster_rcnn\banana_77.jpg
Saved: ../test_results/faster_rcnn\bana